In [1]:
%matplotlib inline

In [2]:
import os
import pathlib
import sys

In [3]:
base_path = pathlib.Path(os.getcwd())
base_path = str(base_path.parent)
sys.path = [base_path] + sys.path

In [4]:
import glob
import random as python_random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.utils import model_to_dot
from IPython.display import SVG

In [6]:
sns.set(context="notebook", style="darkgrid", palette="deep", font="sans-serif", font_scale=1.0, color_codes=True)

In [7]:
os.makedirs("./img/", exist_ok=True)
os.makedirs("./score/", exist_ok=True)

In [8]:
def set_experimental_environment(seed=6902):
    K.clear_session()

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.
    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.
    python_random.seed(seed)

    # The below set_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see:
    # https://www.tensorflow.org/api_docs/python/tf/random/set_seed
    tf.random.set_seed(seed)

In [9]:
from pynvml import *

try:
    nvmlInit()
    print("Driver Version:", nvmlSystemGetDriverVersion())
    deviceCount = nvmlDeviceGetCount()
    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        print("Device", i, ":", nvmlDeviceGetName(handle))
    nvmlShutdown()
except NVMLError as error:
    print(error)

Driver Version: b'419.17'
Device 0 : b'GeForce GTX 1070 Ti'


In [10]:
from cpuinfo import get_cpu_info

for key, value in get_cpu_info().items():
    print("{0}: {1}".format(key, value))

python_version: 3.6.10.final.0 (64 bit)
cpuinfo_version: [7, 0, 0]
cpuinfo_version_string: 7.0.0
arch: X86_64
bits: 64
count: 12
arch_string_raw: AMD64
vendor_id_raw: GenuineIntel
brand_raw: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz
hz_advertised_friendly: 3.2000 GHz
hz_actual_friendly: 3.1920 GHz
hz_advertised: [3200000000, 0]
hz_actual: [3192000000, 0]
l2_cache_size: 1572864
stepping: 10
model: 158
family: 6
l3_cache_size: 12582912
flags: ['3dnow', '3dnowprefetch', 'abm', 'acpi', 'adx', 'aes', 'apic', 'avx', 'avx2', 'bmi1', 'bmi2', 'clflush', 'clflushopt', 'cmov', 'cx16', 'cx8', 'de', 'dtes64', 'dts', 'erms', 'est', 'f16c', 'fma', 'fpu', 'fxsr', 'hle', 'ht', 'hypervisor', 'ia64', 'invpcid', 'lahf_lm', 'mca', 'mce', 'mmx', 'movbe', 'mpx', 'msr', 'mtrr', 'osxsave', 'pae', 'pat', 'pbe', 'pcid', 'pclmulqdq', 'pdcm', 'pge', 'pni', 'popcnt', 'pse', 'pse36', 'rdrnd', 'rdseed', 'rtm', 'sep', 'serial', 'smap', 'smep', 'ss', 'sse', 'sse2', 'sse4_1', 'sse4_2', 'ssse3', 'tm', 'tm2', 'tsc', 'vme', 

In [11]:
import psutil 

mem = psutil.virtual_memory() 
print("memory: {0:.2f}GB".format(mem.total / 1024**3))

memory: 15.92GB


In [12]:
BAYES_MODELS = [
    "MNIST-CNN",
]

LAST_ACTIVATIONS = [
    "sigmoid",
    "softmax",
]

PREDICTION_MODES = [
    "Normal-mode",
    "Linear-mode",
    "Independent-mode",
    "Upper-mode",
    "MC-mode",
]

DATASETS = [
    "MNIST",
    "Fashion",
    "Kuzushiji",
    "Kannada",
    "EMNIST-MNIST",
]

In [13]:
class Args(object):
    seed = 6902
    train_domain = "Fashion"
    bayes_model= BAYES_MODELS[0]
    num_mc = 2000
    optimizer = "Adam"
    train_batch_size = 128
    test_batch_size = 2048
    max_epochs = 1
    patience = 10
    rhos = [0.0, 1.0e-5, 5.0e-5, 1.0e-4, 5.0e-4, 1.0e-3, 5.0e-3, 1.0e-2, 5.0e-2, 1.0e-1]

In [14]:
def kuzushiji_load_data():
    ds = tfds.load(
        "kmnist", split=["train", "test"], shuffle_files=False, batch_size=-1
    )
    ds = tfds.as_numpy(ds)
    y_train = ds[0]["label"].astype("uint8")
    x_train = ds[0]["image"].reshape((-1, 28, 28)).astype("uint8")
    y_test = ds[1]["label"].astype("uint8")
    x_test = ds[1]["image"].reshape((-1, 28, 28)).astype("uint8")

    return (x_train, y_train), (x_test, y_test)


def kannada_load_data():
    path = tf.keras.utils.get_file(
        "y_kannada_MNIST_train.npz",
        "https://github.com/vinayprabhu/Kannada_MNIST/raw/master/data/output_tensors/MNIST_format/y_kannada_MNIST_train.npz",
    )
    y_train = np.load(path)["arr_0"]

    path = tf.keras.utils.get_file(
        "X_kannada_MNIST_train.npz",
        "https://github.com/vinayprabhu/Kannada_MNIST/raw/master/data/output_tensors/MNIST_format/X_kannada_MNIST_train.npz",
    )
    x_train = np.load(path)["arr_0"]

    path = tf.keras.utils.get_file(
        "y_kannada_MNIST_test.npz",
        "https://github.com/vinayprabhu/Kannada_MNIST/raw/master/data/output_tensors/MNIST_format/y_kannada_MNIST_test.npz",
    )
    y_test = np.load(path)["arr_0"]

    path = tf.keras.utils.get_file(
        "X_kannada_MNIST_test.npz",
        "https://github.com/vinayprabhu/Kannada_MNIST/raw/master/data/output_tensors/MNIST_format/X_kannada_MNIST_test.npz",
    )
    x_test = np.load(path)["arr_0"]

    return (x_train, y_train), (x_test, y_test)


def emnist_mnist_load_data():
    ds = tfds.load(
        "emnist/mnist", split=["train", "test"], shuffle_files=False, batch_size=-1
    )
    ds = tfds.as_numpy(ds)
    y_train = ds[0]["label"].astype("uint8")
    x_train = ds[0]["image"].reshape((-1, 28, 28)).astype("uint8")
    y_test = ds[1]["label"].astype("uint8")
    x_test = ds[1]["image"].reshape((-1, 28, 28)).astype("uint8")

    x_train = np.array([x.T for x in x_train])
    x_test = np.array([x.T for x in x_test])

    return (x_train, y_train), (x_test, y_test)


def load_dataset(dataset, val_size=1.0 / 6.0):
    if dataset == DATASETS[0]:
        load_data = keras.datasets.mnist.load_data
    elif dataset == DATASETS[1]:
        load_data = keras.datasets.fashion_mnist.load_data
    elif dataset == DATASETS[2]:
        load_data = kuzushiji_load_data
    elif dataset == DATASETS[3]:
        load_data = kannada_load_data
    elif dataset == DATASETS[4]:
        load_data = emnist_mnist_load_data
    else:
        raise ValueError("Error")

    # input image dimensions
    num_classes = 10

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    if val_size > 0.0:
        x_train, x_val, y_train, y_val = train_test_split(
            x_train, y_train, test_size=val_size, stratify=y_train
        )
    else:
        x_val, y_val = x_train, y_train

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_val = keras.utils.to_categorical(y_val, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_val, y_val), (x_test, y_test), num_classes

In [15]:
def calc_softmax_entropy(prob):
    entropy = np.sum(-prob * np.log(np.maximum(prob, 1.0e-7)), axis=-1)
    return entropy


def calc_sigmoid_entropy(prob):
    entropy = np.sum(
        -prob * np.log(np.maximum(prob, 1.0e-7))
        - (1.0 - prob) * np.log(np.maximum(1.0 - prob, 1.0e-7)),
        axis=-1,
    )
    return entropy

In [16]:
def create_last_bayes_model(input_shape, output_shape, last_activation):
    # https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
    inputs = keras.Input(input_shape)
    conv1 = layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(inputs)
    conv2 = layers.Conv2D(64, (3, 3), activation="relu")(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    drop1 = layers.Dropout(0.25)(pool1)
    flat1 = layers.Flatten()(drop1)
    dense1 = layers.Dense(128, activation="relu")(flat1)
    drop2 = layers.Dropout(0.5)(dense1)
    dense2 = layers.Dense(output_shape, activation=last_activation)(drop2)
    model = keras.Model(inputs=inputs, outputs=dense2)

    return model


def create_bayes_model(bayes_model_name, input_shape, output_shape, last_activation):
    if bayes_model_name == BAYES_MODELS[0]:
        model = create_last_bayes_model(input_shape, output_shape, last_activation)
    else:
        raise ValueError()

    return model

In [17]:
args = Args()

In [18]:
from vpbnn import vlayers
from vpbnn.models import nn2vpbnn

seed = 6902
last_activation = "sigmoid"

set_experimental_environment(seed)

(x_train, y_train), (x_val, y_val), (x_test, y_test), num_classes = load_dataset(
    args.train_domain
)        

input_shape = x_train.shape[1:]
model = create_bayes_model(
    args.bayes_model, input_shape, num_classes, last_activation
)

if last_activation == "softmax":
    loss_func = "categorical_crossentropy"
    calc_entropy = calc_softmax_entropy
elif last_activation == "sigmoid":
    loss_func = "binary_crossentropy"
    calc_entropy = calc_sigmoid_entropy
else:
    raise ValueError()

es = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=args.patience,
    verbose=1,
    mode="auto",
    restore_best_weights=True,
)

model.compile(
    loss=loss_func, optimizer=args.optimizer, metrics=["accuracy"],
)

model.fit(
    x_train,
    y_train,
    batch_size=args.train_batch_size,
    epochs=args.max_epochs,
    verbose=2,
    validation_data=(x_val, y_val),
    callbacks=[es],
)

if last_activation == "sigmoid":
    best_rho = None
    best_ll = -np.inf
    for rho in args.rhos:
        vmodel = nn2vpbnn(model, variance_mode=3, rho=rho)
        y_prob, y_var = vmodel.predict(x_val, batch_size=args.test_batch_size)
        ll = -0.5 * np.square(y_val - y_prob) / np.maximum(y_var, 1.0e-7) - 0.5 * np.log(2.0 * np.pi) - 0.5 * np.log(np.maximum(y_var, 1.0e-7))
        ll = ll.mean()
        print("rho: {0}, ll: {1}".format(rho, ll))                
        if ll > best_ll:
            best_rho = rho
            best_ll = ll
    print("best_rho: {0}, best_ll: {1}".format(best_rho, best_ll))
    vmodel = nn2vpbnn(model, rho=best_rho)
else:
    vmodel = nn2vpbnn(model)

Train on 50000 samples, validate on 10000 samples
50000/50000 - 5s - loss: 0.1220 - accuracy: 0.9533 - val_loss: 0.0663 - val_accuracy: 0.9746
rho: 0.0, ll: 0.8545447587966919
rho: 1e-05, ll: 0.8547410368919373
rho: 5e-05, ll: 0.8555192351341248
rho: 0.0001, ll: 0.8564772009849548
rho: 0.0005, ll: 0.8635912537574768
rho: 0.001, ll: 0.8712509870529175
rho: 0.005, ll: 0.8992081880569458
rho: 0.01, ll: 0.8863914608955383
rho: 0.05, ll: 0.4436076283454895
rho: 0.1, ll: 0.060839131474494934
best_rho: 0.005, best_ll: 0.8992081880569458


In [19]:
def mc_predict(vmodel, num_mc):
    for _ in range(num_mc):
        _ = vmodel.predict(x_test, batch_size=args.test_batch_size)

### VPBNN-Mode (adaptive rho)

In [20]:
for layer in vmodel.layers:
    if isinstance(layer, vlayers.VarianceLayer):
        layer.variance_mode = 3
vmodel.compile(loss=loss_func)
%timeit -r 10 -n 1 vmodel.predict(x_test, batch_size=args.test_batch_size)

175 ms ± 31.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


### MC-Mode (T=10)

In [21]:
for layer in vmodel.layers:
    if isinstance(layer, vlayers.VarianceLayer):
        layer.variance_mode = 4
vmodel.compile(loss=loss_func)
%timeit -r 10 -n 1 mc_predict(vmodel, 10)

1.56 s ± 16.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


### MC-Mode (T=100)

In [22]:
for layer in vmodel.layers:
    if isinstance(layer, vlayers.VarianceLayer):
        layer.variance_mode = 4
vmodel.compile(loss=loss_func)
%timeit -r 10 -n 1 mc_predict(vmodel, 100)

15.6 s ± 28.3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


### MC-Mode (T=200)

In [23]:
for layer in vmodel.layers:
    if isinstance(layer, vlayers.VarianceLayer):
        layer.variance_mode = 4
vmodel.compile(loss=loss_func)
%timeit -r 10 -n 1 mc_predict(vmodel, 200)

31.3 s ± 13.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


### MC-Mode (T=300)

In [24]:
for layer in vmodel.layers:
    if isinstance(layer, vlayers.VarianceLayer):
        layer.variance_mode = 4
vmodel.compile(loss=loss_func)
%timeit -r 10 -n 1 mc_predict(vmodel, 300)

47 s ± 36.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
